In [1]:
from bs4 import BeautifulSoup
import re
import requests


url = 'https://today.line.me/tw/v2/tab'
res1 = requests.get(url)
soup = BeautifulSoup(res1.text, 'lxml')
pattern = re.compile(r'"[a-f0-9]{8}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{12}"')
uuids = [x[1:-1] for x in pattern.findall(str(soup.body.find('script')))]
items = []

for uuid in uuids:
    apiurl = "https://today.line.me/api/v6/listings/{}?offset=0&length=20&country=tw".format(uuid)
    res2 = requests.get(apiurl)
    data = res2.json()
    for i in data['items']:
        item = {
            'title': i['title'],
            'publisher': i.get('publisher'),
            'publishTimeUnix': i.get('publishTimeUnix', 0),
            'url': "https://today.line.me/tw/v2/article/{}".format(i['url']['hash']) if i['url'].get('hash') else i['url']['url']
        }
        items.append(item)

In [2]:
import pandas as pd


df = pd.DataFrame(items)
df.to_csv('output.csv', index=False)

In [3]:
df

,title,publisher,publishTimeUnix,url
0,積少成多 居家省水這樣做,大愛新聞影音,1617891260000,https://today.line.me/tw/v2/article/8YBBan
1,腳趾頭又大又長 連走路都是煎熬,TomoNews,1617926400000,https://today.line.me/tw/v2/article/yKJVM6
2,統聯客運走錯地下道 車頭削一半,民視新聞影音,1617917703000,https://today.line.me/tw/v2/article/oBazO6
3,臺安醫院涉詐領補助費 院長80萬交保,華視影音,1617929040000,https://today.line.me/tw/v2/article/YzqewQ
4,台鐵員工月台敬禮 全線120列車不捨鳴笛,民視新聞影音,1617852421000,https://today.line.me/tw/v2/article/ENEDqm
...,...,...,...,...
291,宅品味▷微波爐8招聰明做菜 還能幫廚房用品殺菌,康健雜誌,1617258060000,https://today.line.me/tw/v2/article/30KPkv
292,時尚▷跟上鳳梨熱潮！用開架彩妝打造3款黃色系眼妝,Marie Claire美麗佳人,1617753600000,https://today.line.me/tw/v2/article/lymzNG
293,新聞台▷麻辣評論針砭新聞時事 鎖定民視辣新聞,民視新聞台,1591768500000,https://today.line.me/tw/v2/article/PNNoG5
294,電影▷奧斯卡最強黑馬《夢想之地》5大亮點搶先看,Tatler Taiwan,1617688080000,https://today.line.me/tw/v2/article/RlEQaq
